# DSWE Analysis

This code is used to analyze DSWE inundation assets generated using the 'Monthly Landsat DSWE Generator'script. The user provides folders to the DSWE and QC folders. It outputs a .csv of statistics including the DSWE pixel and area coverage by value, as well as QC statistics for each QC product. 

DSWE Methodology: Jones, J.W., 2019. Improved Automated Detection of Subpixel-Scale Inundation—Revised Dynamic Surface Water Extent (DSWE) Partial Surface Water Tests. Remote Sensing 11, 374. https://doi.org/10.3390/rs11040374

Landsat Collection 2: Earth Resources Observation and Science (EROS) Center. (2020). Landsat 8-9 Operational Land Imager / Thermal Infrared Sensor Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9OGBGM6. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 7 Enhanced Thematic Mapper Plus Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9C7I13B. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 4-5 Thematic Mapper Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9IAXOVV.

Google Earth Engine: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., Moore, R., 2017. Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, Big Remotely Sensed Data: tools, applications and experiences 202, 18–27. https://doi.org/10.1016/j.rse.2017.06.031

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: August 18, 2025

## Import packages

In [23]:
import ee
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

ee.Initialize()

## Initialize functions

In [41]:
def generate_monthly_ids(folder_prefix, prefix_label, start_year, start_month, end_year, end_month):
    start_date = datetime(start_year, start_month, 1)
    end_date = datetime(end_year, end_month, 1)
    image_ids = []

    while start_date <= end_date:
        year = start_date.year
        month = f"{start_date.month:02d}"
        img_id = f"{folder_prefix}/{prefix_label}_{year}_{month}"
        date_str = f"{year}-{month}"
        image_ids.append((img_id, date_str))
        start_date += relativedelta(months=1)

    return image_ids

def compute_dswe_stats(image_id, date_str, scale=30):
    stats = {'date': date_str}
    classes = [0, 1, 2, 3, 4]

    try:
        image = ee.Image(image_id).select('dswe')

        for cls in classes:
            count = image.eq(cls).reduceRegion(
                reducer=ee.Reducer.sum(),
                scale=scale,
                maxPixels=1e13
            ).get('dswe')
            area_km2 = ee.Number(count).multiply(scale * scale).divide(1e6)
            stats[f'count_{cls}'] = count.getInfo() if count else 0
            stats[f'area_km2_{cls}'] = area_km2.getInfo() if area_km2 else 0

        water_mask = image.gt(0)
        water_count = water_mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=scale,
            maxPixels=1e13
        ).get('dswe')
        water_area = ee.Number(water_count).multiply(scale * scale).divide(1e6)

        stats['count_1to4'] = water_count.getInfo() if water_count else 0
        stats['area_km2_1to4'] = water_area.getInfo() if water_area else 0

        return stats

    except Exception:
        print(f"⚠️ DSWE image not found or failed to process: {image_id}")
        return None

def compute_qc_stats(qc_image_id, scale=30):
    try:
        image = ee.Image(qc_image_id).select('expansion_mask')
    except Exception:
        print(f"⚠️ QC image not found or failed to process: {qc_image_id}")
        return {}

    stats = {}

    for value in range(7):  # QC values 0–6
        mask = image.eq(value)
        count = mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=scale,
            maxPixels=1e13
        ).get('expansion_mask')
        area_km2 = ee.Number(count).multiply(scale * scale).divide(1e6)
        stats[f'qc_count_{value}'] = count.getInfo() if count else 0
        stats[f'qc_area_km2_{value}'] = area_km2.getInfo() if area_km2 else 0

    # Total stats
    pixel_count = image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=scale,
        maxPixels=1e13
    ).get('expansion_mask')

    mean_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=scale,
        maxPixels=1e13
    ).get('expansion_mask')

    total_area = ee.Number(pixel_count).multiply(scale * scale).divide(1e6)

    stats['qc_pixel_count'] = pixel_count.getInfo() if pixel_count else 0
    stats['qc_area_km2'] = total_area.getInfo() if total_area else 0
    stats['qc_avg_value'] = mean_value.getInfo() if mean_value else None

    return stats

def batch_process_dswe_with_qc(dswe_folder, qc_folder, start_year, start_month, end_year, end_month, output_csv_path):
    image_entries = generate_monthly_ids(dswe_folder, 'DSWE', start_year, start_month, end_year, end_month)
    all_stats = []

    for dswe_id, date_str in image_entries:
        print(f"📅 Processing {date_str}...")
        dswe_stats = compute_dswe_stats(dswe_id, date_str)
        if dswe_stats:
            year, month = date_str.split('-')
            qc_id = f"{qc_folder}/QC_{year}_{month}"
            qc_stats = compute_qc_stats(qc_id)
            all_stats.append(dswe_stats | qc_stats)

    df = pd.DataFrame(all_stats)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
    df = df.sort_values(by='date')

    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ CSV saved to: {output_csv_path}")

## Execute code for date range

In [42]:
batch_process_dswe_with_qc(
    dswe_folder='projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products',
    qc_folder='projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/QC_Masks',
    start_year=1984,
    start_month=6,
    end_year=2025,
    end_month=7,
    output_csv_path=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Extents\dswe_monthly_summary_08182025.csv'
)


📅 Processing 1984-06...
📅 Processing 1984-07...
📅 Processing 1984-08...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_08
📅 Processing 1984-09...
📅 Processing 1984-10...
📅 Processing 1984-11...
📅 Processing 1984-12...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1984_12
📅 Processing 1985-01...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1985_01
📅 Processing 1985-02...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1985_02
📅 Processing 1985-03...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1985_03
📅 Processing 1985-04...
📅 Processing 1985-05...
⚠️ DSWE

📅 Processing 1991-08...
📅 Processing 1991-09...
📅 Processing 1991-10...
📅 Processing 1991-11...
📅 Processing 1991-12...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1991_12
📅 Processing 1992-01...
📅 Processing 1992-02...
📅 Processing 1992-03...
📅 Processing 1992-04...
📅 Processing 1992-05...
📅 Processing 1992-06...
📅 Processing 1992-07...
📅 Processing 1992-08...
📅 Processing 1992-09...
📅 Processing 1992-10...
📅 Processing 1992-11...
📅 Processing 1992-12...
📅 Processing 1993-01...
📅 Processing 1993-02...
📅 Processing 1993-03...
📅 Processing 1993-04...
📅 Processing 1993-05...
📅 Processing 1993-06...
📅 Processing 1993-07...
📅 Processing 1993-08...
📅 Processing 1993-09...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_1993_09
📅 Processing 1993-10...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/ass

📅 Processing 2005-03...
📅 Processing 2005-04...
📅 Processing 2005-05...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2005_05
📅 Processing 2005-06...
📅 Processing 2005-07...
📅 Processing 2005-08...
📅 Processing 2005-09...
📅 Processing 2005-10...
📅 Processing 2005-11...
📅 Processing 2005-12...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2005_12
📅 Processing 2006-01...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2006_01
📅 Processing 2006-02...
📅 Processing 2006-03...
📅 Processing 2006-04...
📅 Processing 2006-05...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2006_05
📅 Processing 2006-06...
📅 Processing 2006-07...
📅 Processing 2006-08...
📅 Pr

⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2012_10
📅 Processing 2012-11...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2012_11
📅 Processing 2012-12...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2012_12
📅 Processing 2013-01...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2013_01
📅 Processing 2013-02...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products/DSWE_2013_02
📅 Processing 2013-03...
📅 Processing 2013-04...
📅 Processing 2013-05...
📅 Processing 2013-06...
📅 Processing 2013-07...
📅 Processing 2013-08...
📅 Processing 2013-09...
📅 Processing 2013-10...
📅 Proce